In [1]:
import numpy as np
import torch

/Users/beliakovsi/Documents/MIPT/grade/env3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BlenderbotSmallForConditionalGeneration


In [39]:
from transformers import BlenderbotSmallConfig



In [4]:
mname = "facebook/blenderbot_small-90M"
model = BlenderbotSmallForConditionalGeneration.from_pretrained(mname)
model.max_new_tokens = 1000
tokenizer = AutoTokenizer.from_pretrained(mname)
UTTERANCE = "My friends are cool but they eat too many carbs."
print("Human: ", UTTERANCE)

inputs = tokenizer([UTTERANCE], return_tensors="pt")
reply_ids = model.generate(**inputs)
print("Bot: ", tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

REPLY = "I'm not sure"
print("Human: ", REPLY)

NEXT_UTTERANCE = (
    "My friends are cool but they eat too many carbs.</s> <s>what kind of carbs do they eat? "
    "i don't know much about carbs</s> "
    "<s> I'm not sure."
)
inputs = tokenizer([NEXT_UTTERANCE], return_tensors="pt")

print(inputs)

next_reply_ids = model.generate(**inputs)
print("Bot: ", tokenizer.batch_decode(next_reply_ids, skip_special_tokens=True)[0])

Human:  My friends are cool but they eat too many carbs.


/Users/beliakovsi/Documents/MIPT/grade/env3.9/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Bot:  what kind of carbs do they eat? i don't know much about carbs.
Human:  I'm not sure
{'input_ids': tensor([[  42,  643,   46, 1430,   45,   52, 1176,  146,  177,  753, 2430,    3,
            3,    3,  330, 1360,    3,  330,    3,   44,  444,   10,  753, 2430,
           59,   52, 1176,   20,   14,   67,    8,   30,   70,  165,   72,  753,
          372,  330,    3,    3,  330, 1360,    3,  330, 1360,   14,    8,   58,
           48,  225,    5]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}
Bot:  they eat a lot of carbs. carbs are high in fat, protein, and carbohydrates.


In [67]:
from transformers import AutoTokenizer, BlenderbotForConditionalGeneration


class dialogGenerator:
    def __init__(self, mname="facebook/blenderbot-400M-distill"):
        self.mname = mname
        #self.config = BlenderbotSmallConfig(max_new_tokens=1000)
        self.model = BlenderbotForConditionalGeneration.from_pretrained(mname)
        self.tokenizer = AutoTokenizer.from_pretrained(mname)
        self.model.max_new_tokens = 1000 # TODO
        self.model.max_length = 1000
        self.context = []
        self.message_separator = "</s> <s>"
        return
    
    def clear_history(self):
        self.context = []
        return
    
    def message(self, next_message=None):
        if next_message is not None:
            self.context.append(next_message)

        previous_dialog = self.message_separator.join(self.context)

        inputs = self.tokenizer([previous_dialog], return_tensors="pt")
        #print(inputs)
        reply_ids = self.model.generate(max_length=1000, **inputs)
        self.context.append(self.tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

        return self.context[-1]
    
    def print_dialog(self):
        return self.message_separator.join(self.context)
    


In [60]:
dialog_generator = dialogGenerator("facebook/blenderbot-400M-distill")


In [61]:
N = 3
#print(dialog_generator.message(next_message))
for i in range(N):
    next_message = input()
    print(next_message)
    print(dialog_generator.message(next_message))
    

Hi
{'input_ids': tensor([[4424,    2]]), 'attention_mask': tensor([[1, 1]])}
 Hi! How are you? I just got back from walking my dog. Do you have any pets?
I'm fine thanl you. Where are you from?
{'input_ids': tensor([[4424,    2,  228,    1, 4424,    8,  855,  366,  304,   38,  281,  404,
          660,  665,  482, 3568,  395, 1784,   21,  946,  304,  360,  463,  286,
         1272,   38,    2,  228,    1,  281,  476, 1435,  546,   83,  304,   21,
         2354,  366,  304,  482,   38,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
 I am from the united states. I have a dog and a cat. What about you?
NICE!
{'input_ids': tensor([[4424,    2,  228,    1, 4424,    8,  855,  366,  304,   38,  281,  404,
          660,  665,  482, 3568,  395, 1784,   21,  946,  304,  360,  463,  286,
         1272,   38,    2,  228,    1,  281,  476, 1435,  546,   83,  304,   21,
   

In [8]:
dialog_generator.start_new_dialog()

In [9]:
dialog_generator.print_dialog()

''

In [10]:
data = ["some", "data", "lots", "of", "strings"]
separator = "</s> <s>"


print(separator.join(data))

some</s> <s>data</s> <s>lots</s> <s>of</s> <s>strings


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    #print(chat_history_ids)
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're being serious or not, but I'm not sure either.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're being serious or not, but I'm not sure either.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're being serious or not, but I'm not sure either.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're being serious or not, but I'm not sure either.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're being serious or not, but I'm not sure either.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/Users/beliakovsi/Documents/MIPT/grade/env3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
class dialogGeneratorM:
    def __init__(self, mname="microsoft/DialoGPT-medium"):
        self.mname = mname
    
    
        self.tokenizer = AutoTokenizer.from_pretrained(mname, padding_side='right')
        self.model = AutoModelForCausalLM.from_pretrained(mname)
    
        self.chat_history_ids = None
    
        # self.message_separator = "</s> <s>"
   
        return
    
    def clear_history(self):
        self.chat_history_ids = None
        return
    
    def update_history(self, history: list[str]):
        for message in history:
            new_user_input_ids = self.tokenizer.encode(message + self.tokenizer.eos_token, return_tensors='pt')
            self.chat_history_ids = torch.cat([self.chat_history_ids, new_user_input_ids], dim=-1) if self.chat_history_ids is not None else new_user_input_ids
        print(self.chat_history_ids)
        return 
    
    def message(self, next_message=None):
        if (next_message == None):
            print("JOPA")
            return
            
        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = self.tokenizer.encode(next_message + self.tokenizer.eos_token, return_tensors='pt')

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([self.chat_history_ids, new_user_input_ids], dim=-1) if self.chat_history_ids is not None else new_user_input_ids

        # generated a response while limiting the total chat history to 1000 tokens, 
        self.chat_history_ids = self.model.generate(bot_input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)

        return self.tokenizer.decode(self.chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
        # pretty print last ouput tokens from bot
        print("DialoGPT: {}".format(self.tokenizer.decode(self.chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


    
    def print_dialog(self):
        return self.message_separator.join(self.context)

In [19]:
dialog_generator_test = dialogGeneratorM()
dialog_generator_test.update_history(['Hi', 'Hi! :D', 'How are you?', 'I\'m good, how are you?'])


tensor([[17250, 50256, 17250,     0,  1058,    35, 50256,  2437,   389,   345,
            30, 50256,    40,  1101,   922,    11,   703,   389,   345,    30,
         50256]])


In [51]:
dialog_generator = dialogGeneratorM()
#dialog_generator.update_history(['Hi! my name Alice, tell me about yourself'])
N = 3
#print(dialog_generator.message(next_message))
for i in range(N):
    next_message = input()
    print(next_message)
    print(dialog_generator.message(next_message))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


hi alice, i'm sam. i've always wanted to go to the beach. what about you?
I'm a student.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


g
I'm a student too.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


d
I'm a student too.


In [65]:
class BotsDialogGenerator:
    def __init__(self, speaker_a, speaker_b):
        self.speaker_a = speaker_a
        self.speaker_b = speaker_b
        
        self.dialog = []

        return
    
    def create_dialog(self, max_len=5, starting_message="Hi! my name Alice, tell me about yourself") -> list[str]:
        self.speaker_a.clear_history()
        self.speaker_b.clear_history()
        
        self.speaker_b.update_history([starting_message])
        next_message = starting_message
        self.dialog.append(next_message)
        
        for dialog_len in range(max_len):
            response = self.speaker_a.message(next_message)
            self.dialog.append(response)

            next_message = self.speaker_b.message(response)
            self.dialog.append(next_message)
            
        return self.dialog


In [70]:
dialog_bots = BotsDialogGenerator(dialogGenerator(), dialogGeneratorM())

print(dialog_bots.create_dialog(max_len=3))

tensor([[17250,     0,   616,  1438, 14862,    11,  1560,   502,   546,  3511,
         50256]])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['Hi! my name Alice, tell me about yourself', " Hi, my name is samantha. I'm a college student. How about you?", "Hi! I'm a college student too!", " Nice to meet you! What are you studying? I'm studying to be a nurse.", "I'm studying to be a nurse too!", " That's great! Do you have any hobbies? I like to knit and crochet.", "I do have a few hobbies, but I'm not really into them. I'm a bit of a nerd."]
